Load the Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Model ResNet

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [ ]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # ------------------------ 3 ------------------------------------

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    # -------------------------------------- 3 --------------------------------------------

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [ ]:
def ResNet30(input_shape = (288, 432, 3), classes = 5):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    #X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    #X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D( pool_size=(2, 2),name = "avg_pool")(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet30')

    return model

In [ ]:
model = ResNet30(input_shape = (288, 432, 3), classes = 5)
model.summary()

Model: "ResNet30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 288, 432, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 294, 438, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 144, 216, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 144, 216, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Make the numpy array

In [ ]:
import numpy as np
import cv2
import os

instances_train = []
labels_train = []

# Load in the images
for filepath in os.listdir(r"/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train"):
    print(filepath)
    for filename in os.listdir('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train/{0}'.format(filepath)):
      if filename.endswith(".png"):
        #print(filename)
        instances_train.append(cv2.imread('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train/{}/{}'.format(filepath,filename),1))
        #print('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train/{one}/{two}'.format(one = filepath,two = filename))
        if (filepath == "belly_pain"):
          labels_train.append(0)
        elif (filepath == "burping" ):
          labels_train.append(1)
        elif (filepath == "discomfort" ):
          labels_train.append(2)
        elif (filepath == "hungry" ):
          labels_train.append(3)
        elif (filepath == "tired" ):
          labels_train.append(4)

instances_test = []
labels_test = []

for filepath in os.listdir(r"/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/test"):
    print(filepath)
    for filename in os.listdir('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/test/{0}'.format(filepath)):
      if filename.endswith(".png"):
        #print(filename)
        instances_test.append(cv2.imread('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/test/{}/{}'.format(filepath,filename),1))
        #print('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train/{one}/{two}'.format(one = filepath,two = filename))
        if (filepath == "belly_pain"):
          labels_test.append(0)
        elif (filepath == "burping" ):
          labels_test.append(1)
        elif (filepath == "discomfort" ):
          labels_test.append(2)
        elif (filepath == "hungry" ):
          labels_test.append(3)
        elif (filepath == "tired" ):
          labels_test.append(4)


instances_val = []
labels_val = []

for filepath in os.listdir(r"/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/val"):
    print(filepath)
    for filename in os.listdir('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/val/{0}'.format(filepath)):
      if filename.endswith(".png"):
        #print(filename)
        instances_val.append(cv2.imread('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/val/{}/{}'.format(filepath,filename),1))
        #print('/content/drive/My Drive/Baby Cry/donateacry-corpus-master/03-06-2020 Image Files/train/{one}/{two}'.format(one = filepath,two = filename))
        if (filepath == "belly_pain"):
          labels_val.append(0)
        elif (filepath == "burping" ):
          labels_val.append(1)
        elif (filepath == "discomfort" ):
          labels_val.append(2)
        elif (filepath == "hungry" ):
          labels_val.append(3)
        elif (filepath == "tired" ):
          labels_val.append(4)



burping
belly_pain
tired
discomfort
hungry
tired
belly_pain
burping
hungry
discomfort
belly_pain
hungry
tired
discomfort
burping


In [ ]:
X_train_orig = np.array(instances_train)
print(X_train_orig.shape)
X_test_orig = np.array(instances_test)
print(X_test_orig.shape)
X_val_orig = np.array(instances_val)
print(X_val_orig.shape)

Y_train_orig = np.array(labels_train)

Y_test_orig = np.array(labels_test)

Y_val_orig = np.array(labels_val)

print(Y_train_orig.shape)

print(Y_test_orig.shape)

print(Y_val_orig.shape)

(497, 288, 432, 3)
(171, 288, 432, 3)
(174, 288, 432, 3)
(497,)
(171,)
(174,)


make Training set instance_train + instance_val


In [ ]:
X_train_orig = np.concatenate((X_train_orig,X_val_orig))
Y_train_orig = np.concatenate((Y_train_orig,Y_val_orig))

In [ ]:
#X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
from numpy import argmax
from keras.utils import to_categorical

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = to_categorical(Y_train_orig)
Y_test = to_categorical(Y_test_orig)
#print(Y_train[100])
#print(argmax(Y_train[100]))
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

Using TensorFlow backend.


number of training examples = 671
number of test examples = 171
X_train shape: (671, 288, 432, 3)
Y_train shape: (671, 5)
X_test shape: (171, 288, 432, 3)
Y_test shape: (171, 5)


In [ ]:
model.fit(X_train, Y_train, epochs = 50, batch_size = 32)

Epoch 1/50
671/671 [==============================] - 68s 102ms/step - loss: 14.7965 - accuracy: 0.3472
Epoch 2/50
671/671 [==============================] - 46s 69ms/step - loss: 1.6612 - accuracy: 0.4009
Epoch 3/50
671/671 [==============================] - 46s 69ms/step - loss: 1.4898 - accuracy: 0.3949
Epoch 4/50
671/671 [==============================] - 46s 69ms/step - loss: 1.3873 - accuracy: 0.4486
Epoch 5/50
671/671 [==============================] - 46s 69ms/step - loss: 1.3289 - accuracy: 0.4844
Epoch 6/50
671/671 [==============================] - 46s 69ms/step - loss: 1.3822 - accuracy: 0.4188
Epoch 7/50
671/671 [==============================] - 46s 69ms/step - loss: 1.2789 - accuracy: 0.4873
Epoch 8/50
671/671 [==============================] - 46s 69ms/step - loss: 1.3568 - accuracy: 0.5246
Epoch 9/50
671/671 [==============================] - 46s 69ms/step - loss: 1.3394 - accuracy: 0.4844
Epoch 10/50
671/671 [==============================] - 46s 69ms/step - loss: 1.2

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

171/171 [==============================] - 5s 28ms/step
Loss = 2.0368398950811017
Test Accuracy = 0.6666666865348816


--------------------------------------------------------------------------------

# Ignore the next model. VGG model not working yet

## Model VGG

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
"""
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="test", target_size=(224,224))
"""

'\ntrdata = ImageDataGenerator()\ntraindata = trdata.flow_from_directory(directory="data",target_size=(224,224))\ntsdata = ImageDataGenerator()\ntestdata = tsdata.flow_from_directory(directory="test", target_size=(224,224))\n'

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(288,432,3),filters=16,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=16,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 288, 432, 16)      448       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 288, 432, 16)      2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 144, 216, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 144, 216, 32)      4640      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 144, 216, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 72, 108, 32)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 108, 64)      

In [ ]:
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1024,activation="relu"))
model.add(Dense(units=5, activation="softmax"))

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 288, 432, 16)      448       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 288, 432, 16)      2320      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 144, 216, 16)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 144, 216, 32)      4640      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 144, 216, 32)      9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 72, 108, 32)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 108, 64)      

In [ ]:
"""
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])
"""

'\nfrom keras.callbacks import ModelCheckpoint, EarlyStopping\ncheckpoint = ModelCheckpoint("vgg16_1.h5", monitor=\'val_acc\', verbose=1, save_best_only=True, save_weights_only=False, mode=\'auto\', period=1)\nearly = EarlyStopping(monitor=\'val_acc\', min_delta=0, patience=20, verbose=1, mode=\'auto\')\nhist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])\n'

In [ ]:
model.fit(X_train, Y_train, epochs = 50, batch_size = 16)

Epoch 1/50
671/671 [==============================] - 24s 36ms/step - loss: 1.3912 - accuracy: 0.4694
Epoch 2/50
671/671 [==============================] - 21s 31ms/step - loss: 1.3893 - accuracy: 0.4694
Epoch 3/50
671/671 [==============================] - 21s 31ms/step - loss: 1.3911 - accuracy: 0.4694
Epoch 4/50
671/671 [==============================] - 21s 31ms/step - loss: 1.3881 - accuracy: 0.4694
Epoch 5/50
671/671 [==============================] - 21s 31ms/step - loss: 1.3857 - accuracy: 0.4694
Epoch 6/50
671/671 [==============================] - 21s 31ms/step - loss: 1.4001 - accuracy: 0.4694
Epoch 7/50
224/671 [=========>....................] - ETA: 14s - loss: 1.4297 - accuracy: 0.4330

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Train

Now Test